In [ ]:
import pandas as pd
import numpy as np

import sys, os
sys.path.append(os.path.abspath("../src"))

import dataanalyse
import importlib
importlib.reload(dataanalyse)


<module 'dataanalyse' from 'c:\\Users\\johan\\Documents\\Mappe\\src\\dataanalyse.py'>

In [206]:
#Temperaturdata
statistikk_temp, df_total, df = dataanalyse.analyser_fil("../data/temp_gloshaugen_historisk_renset_ 50.csv",datokolonne="tidspunkt",groupby="måned")

for navn, df in statistikk_temp.items():
    print(f"\nSamlet statistikk for: {navn}")

    if 'år' in df.columns:
        display(df.round(2))

    if 'år' in df.columns and 'måned' in df.columns:
        print(f"Årlig gjennomsnitt (kun hele år) for: {navn}")

        # Finn hvor mange måneder hvert år har og tar kun med de som har 12 måneder
        antall_mnd = df.groupby('år')['måned'].count().reset_index(name='antall_måneder')
        hele_år = antall_mnd[antall_mnd['antall_måneder'] == 12]['år']

        df_hele_år = df[df['år'].isin(hele_år)]
        årlig_snitt = df_hele_år.groupby('år')['gjennomsnitt'].mean().reset_index(name='årsgjennomsnitt')

        display(årlig_snitt.round(2))


Analyse av:../data/temp_gloshaugen_historisk_renset_ 50.csv
-------------------------------------------------

Samlet statistikk for: temperatur


,år,måned,gjennomsnitt,median,standardavvik
0,1976,1,-4.55,-4.10,4.65
1,1976,2,-2.26,-1.90,4.53
2,1976,3,-1.20,-1.30,4.40
3,1976,4,5.57,5.55,3.81
4,1976,5,12.56,12.50,4.65
...,...,...,...,...,...
587,2024,12,1.42,1.70,3.46
588,2025,1,-1.39,-0.90,4.10
589,2025,2,-0.99,-0.85,3.84
590,2025,3,4.08,4.15,4.15


Årlig gjennomsnitt (kun hele år) for: temperatur


,år,årsgjennomsnitt
0,1976,6.46
1,1977,6.26
2,1978,5.75
3,1979,5.02
4,1980,5.73
5,1981,5.44
6,1982,6.53
7,1983,7.30
8,1984,6.96
9,1985,4.67


In [207]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook

# Aktiver visning i Jupyter Notebook (hvis du bruker det)
output_notebook()

# Gjør klar data til Bokeh
source = ColumnDataSource(årlig_snitt)

p = figure(
    title="Årsgjennomsnitt temperatur",
    x_axis_label='År', 
    y_axis_label='Temperatur (°C)',
    width=700,
    height=400
)


p.line(x='år', y='årsgjennomsnitt', source=source, line_width=2)
p.circle(x='år', y='årsgjennomsnitt', source=source, size=5)

# Vis graf
show(p)


Loading BokehJS ...

In [208]:
# Beregn endring i gjennomsnittstemperatur fra første til siste år
første = årlig_snitt.iloc[0]
siste = årlig_snitt.iloc[-1]
endring = siste['årsgjennomsnitt'] - første['årsgjennomsnitt']
prosent = (endring / abs(første['årsgjennomsnitt'])) * 100

print(f"Endring i temperatur fra {int(første['år'])} til {int(siste['år'])}:")
print(f"{endring:.2f} °C ({prosent:.2f} %)")


Endring i temperatur fra 1976 til 2024:
1.23 °C (19.09 %)


In [209]:
varmest = årlig_snitt.loc[årlig_snitt['årsgjennomsnitt'].idxmax()]
kaldest = årlig_snitt.loc[årlig_snitt['årsgjennomsnitt'].idxmin()]

print(f"Varmeste år: {int(varmest['år'])} med {varmest['årsgjennomsnitt']:.2f} °C")
print(f"Kaldeste år: {int(kaldest['år'])} med {kaldest['årsgjennomsnitt']:.2f} °C")


Varmeste år: 2020 med 8.76 °C
Kaldeste år: 1985 med 4.67 °C


In [210]:
# Gjenomsnittet for hvert tiår
årlig_snitt['tiår'] = (årlig_snitt['år'] // 10) * 10
tiårs_snitt = årlig_snitt.groupby('tiår')['årsgjennomsnitt'].mean().reset_index()

display(tiårs_snitt.round(2))


,tiår,årsgjennomsnitt
0,1970,5.87
1,1980,6.21
2,1990,6.60
3,2000,7.11
4,2010,7.11
5,2020,7.69


In [211]:
# Temperaturavvik per år
total_snitt = årlig_snitt['årsgjennomsnitt'].mean()
årlig_snitt['avvik'] = årlig_snitt['årsgjennomsnitt'] - total_snitt

display(årlig_snitt[['år', 'årsgjennomsnitt', 'avvik']].round(2))


,år,årsgjennomsnitt,avvik
0,1976,6.46,-0.33
1,1977,6.26,-0.52
2,1978,5.75,-1.03
3,1979,5.02,-1.76
4,1980,5.73,-1.05
5,1981,5.44,-1.34
6,1982,6.53,-0.25
7,1983,7.30,0.52
8,1984,6.96,0.18
9,1985,4.67,-2.12


In [212]:
# Kjør analysen
statistikk, df_total, df = dataanalyse.analyser_fil("../data/klimagassutslipp_renset.csv",sep=",",datokolonne="år",groupby="år")

# Finn riktig kolonnenavn for utslipp
kol_utslipp = [kol for kol in df.columns if "utslipp" in kol and "co2" in kol][0]

# Konverter utslipp til tall
df[kol_utslipp] = pd.to_numeric(df[kol_utslipp], errors="coerce")

# Filtrer bort "Alle kilder"
df = df[~df['kilde_(aktivitet)'].str.contains("alle kilder", case=False)]

# Beregn gjennomsnitt og median PER KILDE PER ÅR
if not df.empty and 'kilde_(aktivitet)' in df.columns and 'år' in df.columns:
    stats_per_kilde_år = df.groupby(['kilde_(aktivitet)', 'år'])[kol_utslipp].agg(['mean', 'median']).reset_index()
    stats_per_kilde_år = stats_per_kilde_år.round(2)

    display(stats_per_kilde_år)
else:
    print("Data mangler 'år' eller 'kilde_(aktivitet)' kolonner.")





Analyse av:../data/klimagassutslipp_renset.csv
-------------------------------------------------


,kilde_(aktivitet),år,mean,median
0,1 Olje- og gassutvinning,1990,8254.0,8254.0
1,1 Olje- og gassutvinning,1991,8127.0,8127.0
2,1 Olje- og gassutvinning,1992,8728.0,8728.0
3,1 Olje- og gassutvinning,1993,9257.0,9257.0
4,1 Olje- og gassutvinning,1994,10053.0,10053.0
...,...,...,...,...
267,9 Andre kilder,2019,2398.0,2398.0
268,9 Andre kilder,2020,2341.0,2341.0
269,9 Andre kilder,2021,2294.0,2294.0
270,9 Andre kilder,2022,2242.0,2242.0


In [213]:
kol_utslipp = [kol for kol in df.columns if "utslipp" in kol and "co2" in kol][0]
df[kol_utslipp] = pd.to_numeric(df[kol_utslipp], errors="coerce")

# Gjennomsnitt og median per kilde totalt
if not df.empty and 'kilde_(aktivitet)' in df.columns:
    print("\nGjennomsnitt og median per kilde totalt:")

    stats_per_kilde = df.groupby('kilde_(aktivitet)')[kol_utslipp].agg(['mean', 'median']).round(2)
    stats_per_kilde = stats_per_kilde.sort_values(by='mean', ascending=False)
    
    display(stats_per_kilde)
else:
    print("Ingen data med individuelle kilder å analysere.")



Gjennomsnitt og median per kilde totalt:


,mean,median
kilde_(aktivitet),,
2 Industri og bergverk,14117.50,14322.5
1 Olje- og gassutvinning,12653.09,13559.0
5 Veitrafikk,8881.03,8909.0
"6 Luftfart, sj�fart, fiske, motorredskaper m.m.",6904.62,7103.0
7 Jordbruk,4788.35,4782.5
9 Andre kilder,2778.12,2832.0
4 Oppvarming i andre n�ringer og husholdninger,1726.79,1897.0
3 Energiforsyning,1112.26,759.0


In [214]:

kol_navn = 'utslipp_til_luft_(1_000_tonn_co2-ekvivalenter,_ar5)'

# Hent ut år + utslipp
total = df_total[['år', kol_navn]].copy()
total = total.rename(columns={kol_navn: 'utslipp'})
total = total.sort_values('år')

# Årlig endring
første = total.iloc[0]
siste = total.iloc[-1]
endring = siste['utslipp'] - første['utslipp']
prosent = (endring / første['utslipp']) * 100

print(f"Endring i utslipp fra {int(første['år'])} til {int(siste['år'])}: {endring:.0f} (ca. {prosent:.2f} %)")


Endring i utslipp fra 1990 til 2023: -4694 (ca. -9.14 %)


In [215]:
# Antar at stats_per_kilde_år allerede finnes og ser slik ut:
# ['kilde_(aktivitet)', 'år', 'mean', 'median']

# Lag en ordbok der hver kilde får sin egen tabell
endring_per_kilde = {}

for kilde in stats_per_kilde_år['kilde_(aktivitet)'].unique():
    df_kilde = stats_per_kilde_år[stats_per_kilde_år['kilde_(aktivitet)'] == kilde].copy()
    df_kilde = df_kilde.sort_values('år')

    df_kilde['endring_abs'] = df_kilde['mean'].diff().round(2)
    df_kilde['endring_prosent'] = df_kilde['mean'].pct_change().multiply(100).round(2)

    endring_per_kilde[kilde] = df_kilde

# Vis alle
for kilde, df_k in endring_per_kilde.items():
    print(f"\nUtslippsendring for kilde: {kilde}")
    display(df_k)




Utslippsendring for kilde: 1 Olje- og gassutvinning


,kilde_(aktivitet),år,mean,median,endring_abs,endring_prosent
0,1 Olje- og gassutvinning,1990,8254.0,8254.0,NaN,NaN
1,1 Olje- og gassutvinning,1991,8127.0,8127.0,-127.0,-1.54
2,1 Olje- og gassutvinning,1992,8728.0,8728.0,601.0,7.40
3,1 Olje- og gassutvinning,1993,9257.0,9257.0,529.0,6.06
4,1 Olje- og gassutvinning,1994,10053.0,10053.0,796.0,8.60
5,1 Olje- og gassutvinning,1995,10237.0,10237.0,184.0,1.83
6,1 Olje- og gassutvinning,1996,11139.0,11139.0,902.0,8.81
7,1 Olje- og gassutvinning,1997,11641.0,11641.0,502.0,4.51
8,1 Olje- og gassutvinning,1998,11308.0,11308.0,-333.0,-2.86
9,1 Olje- og gassutvinning,1999,11829.0,11829.0,521.0,4.61



Utslippsendring for kilde: 2 Industri og bergverk


,kilde_(aktivitet),år,mean,median,endring_abs,endring_prosent
34,2 Industri og bergverk,1990,19140.0,19140.0,NaN,NaN
35,2 Industri og bergverk,1991,17620.0,17620.0,-1520.0,-7.94
36,2 Industri og bergverk,1992,15377.0,15377.0,-2243.0,-12.73
37,2 Industri og bergverk,1993,16325.0,16325.0,948.0,6.17
38,2 Industri og bergverk,1994,17275.0,17275.0,950.0,5.82
39,2 Industri og bergverk,1995,16556.0,16556.0,-719.0,-4.16
40,2 Industri og bergverk,1996,17165.0,17165.0,609.0,3.68
41,2 Industri og bergverk,1997,16854.0,16854.0,-311.0,-1.81
42,2 Industri og bergverk,1998,17168.0,17168.0,314.0,1.86
43,2 Industri og bergverk,1999,16960.0,16960.0,-208.0,-1.21



Utslippsendring for kilde: 3 Energiforsyning


...

,kilde_(aktivitet),år,mean,median,endring_abs,endring_prosent
170,"6 Luftfart, sj�fart, fiske, motorredskaper m.m.",1990,5298.0,5298.0,NaN,NaN
171,"6 Luftfart, sj�fart, fiske, motorredskaper m.m.",1991,5109.0,5109.0,-189.0,-3.57
172,"6 Luftfart, sj�fart, fiske, motorredskaper m.m.",1992,5461.0,5461.0,352.0,6.89
173,"6 Luftfart, sj�fart, fiske, motorredskaper m.m.",1993,5664.0,5664.0,203.0,3.72
174,"6 Luftfart, sj�fart, fiske, motorredskaper m.m.",1994,5858.0,5858.0,194.0,3.43
175,"6 Luftfart, sj�fart, fiske, motorredskaper m.m.",1995,6554.0,6554.0,696.0,11.88
176,"6 Luftfart, sj�fart, fiske, motorredskaper m.m.",1996,6960.0,6960.0,406.0,6.19
177,"6 Luftfart, sj�fart, fiske, motorredskaper m.m.",1997,7246.0,7246.0,286.0,4.11
178,"6 Luftfart, sj�fart, fiske, motorredskaper m.m.",1998,7463.0,7463.0,217.0,2.99
179,"6 Luftfart, sj�fart, fiske, motorredskaper m.m.",1999,7526.0,7526.0,63.0,0.84



Utslippsendring for kilde: 7 Jordbruk


,kilde_(aktivitet),år,mean,median,endring_abs,endring_prosent
204,7 Jordbruk,1990,5050.0,5050.0,NaN,NaN
205,7 Jordbruk,1991,4984.0,4984.0,-66.0,-1.31
206,7 Jordbruk,1992,4956.0,4956.0,-28.0,-0.56
207,7 Jordbruk,1993,4937.0,4937.0,-19.0,-0.38
208,7 Jordbruk,1994,4944.0,4944.0,7.0,0.14
209,7 Jordbruk,1995,4988.0,4988.0,44.0,0.89
210,7 Jordbruk,1996,5021.0,5021.0,33.0,0.66
211,7 Jordbruk,1997,4962.0,4962.0,-59.0,-1.18
212,7 Jordbruk,1998,4954.0,4954.0,-8.0,-0.16
213,7 Jordbruk,1999,4947.0,4947.0,-7.0,-0.14



Utslippsendring for kilde: 9 Andre kilder


,kilde_(aktivitet),år,mean,median,endring_abs,endring_prosent
238,9 Andre kilder,1990,3084.0,3084.0,NaN,NaN
239,9 Andre kilder,1991,2953.0,2953.0,-131.0,-4.25
240,9 Andre kilder,1992,2924.0,2924.0,-29.0,-0.98
241,9 Andre kilder,1993,2968.0,2968.0,44.0,1.50
242,9 Andre kilder,1994,2999.0,2999.0,31.0,1.04
243,9 Andre kilder,1995,3042.0,3042.0,43.0,1.43
244,9 Andre kilder,1996,2989.0,2989.0,-53.0,-1.74
245,9 Andre kilder,1997,3066.0,3066.0,77.0,2.58
246,9 Andre kilder,1998,2912.0,2912.0,-154.0,-5.02
247,9 Andre kilder,1999,2891.0,2891.0,-21.0,-0.72


In [216]:
# Filtrer ut "Alle kilder"
df_alle_kilder = df_total[df_total['kilde_(aktivitet)'].str.lower().str.contains("alle kilder")].copy()

# Finn riktig kolonnenavn for utslipp
kol_utslipp = [kol for kol in df_alle_kilder.columns if "utslipp" in kol and "co2" in kol][0]

# Konverter utslipp til tall hvis nødvendig
df_alle_kilder[kol_utslipp] = pd.to_numeric(df_alle_kilder[kol_utslipp], errors="coerce")

# Finn høyeste og laveste år
maks = df_alle_kilder.loc[df_alle_kilder[kol_utslipp].idxmax()]
minst = df_alle_kilder.loc[df_alle_kilder[kol_utslipp].idxmin()]

# Skriv ut resultat
print(f"Høyeste utslipp:{int(maks['år'])} med {int(maks[kol_utslipp])} (1 000 tonn CO₂-ekv.)")
print(f"Laveste utslipp: {int(minst['år'])} med {int(minst[kol_utslipp])} (1 000 tonn CO₂-ekv.)")


Høyeste utslipp:2007 med 56548 (1 000 tonn CO₂-ekv.)
Laveste utslipp: 2023 med 46654 (1 000 tonn CO₂-ekv.)


In [217]:
# Gjennomsnitt gruppert etter ti-år
df_alle_kilder['tiår'] = (df_alle_kilder['år'] // 10) * 10

# Beregn gjennomsnittlig utslipp per tiår
tiårs_snitt = df_alle_kilder.groupby('tiår')[kol_utslipp].mean().reset_index()
tiårs_snitt[kol_utslipp] = tiårs_snitt[kol_utslipp].round(2)
print(" Gjennomsnittlig utslipp per tiår (1 000 tonn CO₂-ekv.):")
display(tiårs_snitt)


 Gjennomsnittlig utslipp per tiår (1 000 tonn CO₂-ekv.):


,tiår,"utslipp_til_luft_(1_000_tonn_co2-ekvivalenter,_ar5)"
0,1990,51988.90
1,2000,55203.50
2,2010,53432.20
3,2020,48612.75


In [218]:
statistikk_luft, df_total, df = dataanalyse.analyser_fil("../data/gyldig_historisk_luftkvalitet.csv", datokolonne="tid", groupby="år")

for stoff, df in statistikk_luft.items():
    print(f"Statistikk: {stoff}")
    display(df[['år', 'gjennomsnitt']].round(2))



Analyse av:../data/gyldig_historisk_luftkvalitet.csv
-------------------------------------------------
Statistikk: elgeseter_no2_ugm3_hour


c:\Users\johan\Documents\Mappe\src\dataanalyse.py:27: UserWarning: Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df[datokolonne] = pd.to_datetime(df[datokolonne], errors='coerce')


,år,gjennomsnitt
0,2024,16.1


Statistikk: elgeseter_pm10_ugm3_hour


,år,gjennomsnitt
0,2024,17.4


Statistikk: elgeseter_pm2.5_ugm3_hour


,år,gjennomsnitt
0,2024,4.72


In [219]:
df_luft = pd.read_csv("../data/gyldig_historisk_luftkvalitet.csv", sep=",")

df_luft.columns = df_luft.columns.str.strip().str.lower()
df_luft.columns = df_luft.columns.str.replace(" ", "_").str.replace("µg/m³", "ugm3").str.replace("ug/m3", "ugm3")

df_luft["tid"] = pd.to_datetime(df_luft["tid"], errors="coerce")
df_luft["dato"] = df_luft["tid"].dt.date
df_luft["år"] = df_luft["tid"].dt.year
df_luft["måned"] = df_luft["tid"].dt.month
df_luft["time"] = df_luft["tid"].dt.hour

målinger = [kol for kol in df_luft.columns if "ugm3" in kol]

C:\Users\johan\AppData\Local\Temp\ipykernel_18160\1563204585.py:6: UserWarning: Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_luft["tid"] = pd.to_datetime(df_luft["tid"], errors="coerce")


In [220]:
# Analyser hver måling separat
for stoff in målinger:
    print("-------------------------------------------------------------")
    print(f"               Analyse av: {stoff}")
    print("-------------------------------------------------------------")

    # Gjennomsnitt (dag) 
    # Kan brukes hvis relevant i vidre analyse, men blir ikke printet ut her 
    daglig = df_luft.groupby("dato")[stoff].mean().reset_index(name="daglig_snitt")

    # Gjennomsnitt (år)
    årlig = df_luft.groupby("år")[stoff].mean().reset_index(name="årssnitt")
    print("Årlig Gjennomsnitt:")
    display(årlig.round(2))

    # Gjennomsnitt (måned)
    månedlig = df_luft.groupby("måned")[stoff].mean().reset_index(name="månedssnitt")
    print("Månedlig gjennomsnitt:")
    display(månedlig.round(2))

    # Luftkvalitet snitt pr. klokkeslett
    rush = df_luft.groupby("time")[stoff].mean().reset_index(name="snitt_per_klokkeslett")
    print(f"Når på døgnet det er mest {stoff} :")
    display(rush.round(2))

    # Ekstremal verdier 
    maks_rad = df_luft.loc[df_luft[stoff].idxmax()]
    min_rad = df_luft.loc[df_luft[stoff].idxmin()]
    print(f"Ekstremer for {stoff}:")
    print(f"Maks: {maks_rad[stoff]:.2f} µg/m³ @ {maks_rad['tid']}")
    print(f"Min: {min_rad[stoff]:.2f} µg/m³ @ {min_rad['tid']}")

-------------------------------------------------------------
               Analyse av: elgeseter_no2_ugm3_hour
-------------------------------------------------------------
Årlig Gjennomsnitt:


,år,årssnitt
0,2024,16.1


Månedlig gjennomsnitt:


,måned,månedssnitt
0,1,6.58
1,2,27.86
2,3,19.46
3,4,15.68
4,5,15.75
5,6,11.56
6,7,10.21
7,8,12.44
8,9,16.49
9,10,17.63


Når på døgnet det er mest elgeseter_no2_ugm3_hour :


...

Månedlig gjennomsnitt:


,måned,månedssnitt
0,1,2.06
1,2,5.21
2,3,4.97
3,4,4.80
4,5,7.26
5,6,5.23
6,7,3.57
7,8,3.93
8,9,4.41
9,10,3.51


Når på døgnet det er mest elgeseter_pm2.5_ugm3_hour :


,time,snitt_per_klokkeslett
0,0,4.93
1,1,4.31
2,2,3.92
3,3,3.70
4,4,3.30
5,5,3.22
6,6,3.50
7,7,4.04
8,8,4.63
9,9,4.93


Ekstremer for elgeseter_pm2.5_ugm3_hour:
Maks: 37.45 µg/m³ @ 2024-04-16 08:00:00
Min: 0.00 µg/m³ @ 2024-01-31 17:00:00
